# Non-equilibrium Systems, Transport Coefficients, and Coupling Methods

In [ ]:
using LinearAlgebra
using Random
using Plots
using LaTeXStrings
using Statistics
using StatsBase

# I) Non-equilibrium Dynamics and Transport Coefficients

## 1) Overdamped Case
As a first model for non-equilibrium, we consider overdamped Langevin dynamics with potential $U:\mathbb{R}^d \to \mathbb{R}$ and inverse temperature $\beta > 0$ perturbed by a non-gradient vector field $F:\mathbb{R}^d \to \mathbb{R}^d$ with the pertubation modulated by a parameter $\eta \in \mathbb{R}$
$${\rm d}X_t^\eta = \left(-\nabla V\left(X_t^\eta\right) + \eta F\left(X_t^\eta\right)\right)\,{\rm d}t + \sqrt{\frac{2}{\beta}}\,{\rm d}W_t. \tag{Non-equilibrium Dynamics}$$
Under appropriate assumptions on $V$ and $F$ (sufficient contractivity of $-\nabla V$, boundness of $F$, ...), the above dynamics admits unique invariant measure $\nu_\eta$. When $\eta = 0$, $\nu_0$ is the Gibbs measure $$\nu_0(dx) = Z_\beta^{-1} \exp\left(-\beta V(x)\right)dx,$$ and the dynamics of $X^0$ is reversible with respect $\nu_0$. If $\eta \neq 0$, then $\nu_\eta$ is no longer of Gibbs form and the dynamics of $X^\eta$ is not reversible with respect to $\nu_\eta$. 

For an observable $R$, we define the transport coefficient $\alpha_R$ to be linear response at $\eta = 0$ of the expectation of the observable
$$ \alpha_R = \lim_{\eta \to 0} \frac{\nu_\eta(R) - \nu_0(R)}{\eta},$$
i.e. $\nu_\eta(R) \approx \nu_0(R) + \alpha_R \eta$ for $|\eta| \ll 1$.

Suppose that $\nu_0(R) = 0$. We can approximate $\alpha_R$ with a finite difference approximation $\alpha_{R, \eta} = \eta^{-1}\nu_{\eta}(R) = \alpha_R + \mathrm{O}(\eta)$. In practice, we do not have access to $\nu_\eta$ and instead estimate $\nu_\eta(R)$ via a time average. Denote by $\widehat{\Phi}_{\eta, t}$ the standard nonequlibrum molecular dynamics (NEMD) estimator for $\alpha_R$, given by 
$$\widehat{\Phi}_{\eta, T} = \frac{1}{\eta T}\int_0^T R\left(X_t^\eta\right) dt \xrightarrow[\text{a.s.}]{T \to \infty} \alpha_{\eta, R} = \alpha_R + \mathrm{O}(\eta). \tag{NEMD Estimator}$$
The discrete-time version of this estimator is the time average of the Euler-Maruyama discretization in lieu of the continuous-time solution. The statistical error (in continuous-time) of this estimator is dictated by the central limit theorem
$$\sqrt{T}\left(\widehat{\Phi}_{\eta, T} - \alpha_{\eta, R}\right) \xrightarrow[\text{in law}]{T\to \infty} \mathcal{N}\left(0, \frac{\sigma_{R,\eta}^2}{\eta^2}\right).$$
The error of the continuous-time estimator can be summarized as follows
$$\widehat{\Phi}_{\eta, T} = \alpha_R + \text{ Finite Difference Error } + \text{ Finite Integration Time Bias } + \text{ Statistical Error } = \alpha_R + \mathrm{O}(\eta) + \mathrm{O}\left(\frac{1}{\eta T}\right) + \mathrm{O}_P\left(\frac{1}{\eta \sqrt{T}}\right).$$ The error in discrete-time is roughly the same plus an error term due to time discretization.

Our two running examples through this notebook will be a harmonic potential in two dimensions and the entropic switch potential perturbed by a rotational force $F$
$$ F(x) = Jx = \left[ \begin{matrix} 0 & 1\\ -1 & 0\end{matrix} \right]x$$
Observe that this force is non-gradient since $\mathrm{curl}(F) \equiv -2$.

In [ ]:
J = [0. 1
    -1 0]

#Defining the rotational force
rotation(x) = J*x
;

#### i) Harmonic potential (Ornstein-Uhlenbeck process)
As a potential, we take $ U(x) = \frac{x^T A x}{2}$ with
$$ A = \left[ \begin{matrix} 2 & -1\\ -1 & 2\end{matrix} \right].$$
Then the perturbed overdamped Langevin dynamics is given by a linear SDE
$$ {\rm d}X_t^\eta = \left(-A + \eta J\right)X_t^\eta \,{\rm d}t + \sqrt{\frac{2}{\beta}}\,{\rm d}W_t.$$
In this case we can solve everything explicitly. 

<details>
    The invariant measure $\nu_\eta$ is a centered Gaussian with covariance matrix, $\Sigma_{\eta, \beta}$, satisfying
$$ \left(A -\eta J\right)\Sigma_{\eta, \beta} + \Sigma_{\eta, \beta}\left(A -\eta J\right)^T = \frac{2}{\beta}\mathrm{Id}, $$
specifically
$$\Sigma_{\eta, \beta} = \frac{1}{\left(3+\eta^2\right)\beta} \left[\begin{matrix} 2 + (\eta^2 +\eta)/2 & 1\\ 1 & 2 +(\eta^2 - \eta)/2\end{matrix}\right].$$
For $|\eta| < \sqrt{3}$, we can write $\Sigma_{\eta, \beta}$ as a power expansion in $\eta$,
$$\begin{aligned}
\Sigma_{\eta, \beta} &= \frac{1}{3\beta}\left[\begin{matrix} 2 & 1\\ 1 & 2\end{matrix}\right] + \frac{\eta}{6\beta}\left[\begin{matrix} 1 & 0\\ 0 & -1 \end{matrix}\right] + \frac{1}{6\beta}\sum_{k\geq 1} \left(-3\right)^{-k}\left(\eta^{2k}\left[\begin{matrix} 1 & 2\\ 2 & 1\end{matrix}\right] + \eta^{2k+1}\left[\begin{matrix}1 & 0\\ 0 & -1 \end{matrix}\right]\right).
\end{aligned}$$
This expansion indicates that the difference of the variances of the first and second components exhibts a linear response. If we choose as our observable $R(x) = x_1^2 - x_2^2$, then we get that 
$$\nu_\eta\left(R\right) = \frac{\eta}{\left(3+\eta^2\right)\beta} = \frac{\eta}{3\beta} + \frac{1}{3\beta}\sum_{k\geq 1}\left(-3\right)^{-k}\eta^{2k+1},$$
with the sum being absolutely convergent when $|\eta | < \sqrt{3}$. 
</details>

In [ ]:
"""
Plotting the response as a function of η
"""

ν_η_R(η, β) = η/(β*(3 + η^2))
β = 1
plot(0:0.01:1,ν_η_R.(0:0.01:1, β), label = "Response")
plot!([0, 1], [0, 1/(3*β)], color = :black, label = latexstring("Slope at \$\\eta = 0\$"))
plot!(xlabel = L"\eta")

Consequently, the transport coefficient is given by
$$ \alpha_R = \lim_{\eta \to 0} \frac{\nu_\eta(R)}{\eta} = \frac{1}{3\beta}$$

In [ ]:
"""
Defining the harmonic potential and its gradient
"""
A = [2 -1. 
    -1 2]

Harmonic(x) = x'*A*x/2
Harmonic(x,y) = [x, y]' *A *[x,y]/2
∇Harmonic(x) = A*x


"""
Difference of variances response function
"""
function R(x,y)
    return x^2 - y^2
end

R(x::Vector{<:Real}) = R(x...)


"""
Plotting harmonic potential
"""
xlimits = -2., 2.
ylimits = -2., 2.
xrange = range(xlimits..., 200)
yrange = range(ylimits..., 200)
contourf(xrange, yrange, Harmonic, levels=50, cmap=:hsv)

Below we plot a sample trajectory of the dynamics.

In [ ]:
"""
    em_integration_step!(x::Vector{<:Real}, G::Vector{<:Real}, ∇V, F, η, β, dt; RNG =Random.default_rng())

One step Euler-Maruyama discretization of the SDE (Non-eq Dynamics). Configuration x is update in place and then
returned by the function. Noise vector G is also updated in place. The random number generator can be specified
via optional keyword argument RNG.

"""
function em_integration_step!(x::Vector{<:Real}, G::Vector{<:Real}, ∇V, F, η, β, Δt; RNG =Random.default_rng())
    x .+= (.-∇V(x) .+ η*F(x)).* Δt + sqrt(2*Δt/β)*randn!(RNG, G)
    return x
end

"""
   overdamped_trajectory(x_0::Vector{<:Real}, η, β, Δt, ∇V::Function, F::Function; RNG = Random.default_rng(), verbose::Bool=true)

Simulate overdamped trajectory starting from `x_0` with an Euler-Maruyama scheme using time step `Δt` and total 
integration time `T`. The random number generator can be specified via optional keyword argument `RNG`.

Outputs final position and trajectory
"""

function overdamped_trajectory(x_0::Vector{<:Real}, η, β, Δt, T, ∇V::Function, F::Function; RNG = Random.default_rng(), verbose::Bool=true)    
    # Get the number of iterations
    Nit = round(Int, T/Δt)
    Δt = T/Nit
    
    # Frequency of the points to show the progress of the simulations
    nlog = round(Int, Nit/10)

    #Initialize array holding trajectory
    trajectory = Array{Float64, 2}(undef, Nit+1, length(x_0))
    trajectory[1, :] .= x_0

    #Initialize noise vector
    G = Array{Float64, 1}(undef, length(x_0))
    
    #Initial position
    x = copy(x_0)

    if verbose
        print("Integrating using the overdamped dynamics for $(Nit) iterations\t"); flush(stdout)
    end

    for i in 2:(Nit+1)
        
        # Prints a point every nlog iterations
        if verbose && i % nlog == 0
            print("."); flush(stdout)
        end

        # Integrate one step and update trajectory
        trajectory[i, :] .= em_integration_step!(x, G, ∇V, F, η, β, Δt; RNG =RNG)
        
    end
    
    if verbose
        println("\tDone"); flush(stdout)
    end

    return x, trajectory

end

    

#Parameters of the sample trajectory feel free to modify and observe what changes
x_0 = [1., 0]
η = 0.1
β = 2.
Δt = 0.01
T = 10
RNG = Xoshiro(24092023) #fixing RNG

_, x_traj = overdamped_trajectory(x_0, η, β, Δt, T, ∇Harmonic, rotation, RNG = RNG, verbose = false)

contourf(xrange, yrange, Harmonic, levels=50, cmap=:hsv)
plot!(x_traj[:,1], x_traj[:, 2], color=:black, label = "")

In the next cell, we plot a trajectory of the discretized estimator $\widehat{\Phi}_{\eta, T}$.

Vary the total runtime $T$ and strength of the perturbation $\eta$ and comment on the convergence of $\widehat{\Phi}_{\eta, T}$

In [ ]:
x_0 = [1., 0]
η = 1.
β = 2.
Δt = 0.01
T = 5000
RNG = Xoshiro(24092023) #fixing RNG

α_R(β) = 1/(3β) #analytic value of the transport coefficient
α_η_R(β,η) = 1/(β*(3 + η^2)) #analytic value of the finite difference approximation of the transport coefficient

_, x_traj = overdamped_trajectory(x_0, η, β, Δt, T, ∇Harmonic, rotation, RNG = RNG, verbose = false)

phi_hat = cumsum(R.(x_traj[:,1], x_traj[:,2])) ./ axes(x_traj, 1) ./η
plot(0:10Δt:T, phi_hat[1:10:end], label = L"\hat{\Phi}_{\eta, t}")
plot!([0, T], [α_R(β), α_R(β)], label = L"\alpha_R", legendfontsize = 15)
plot!([0, T], [α_η_R(β,η), α_η_R(β,η)], label = L"\alpha_{\eta, R}")
plot!(ylims = (0, 0.5))

What do you observe about the variance and bias of the estimator as a fubction of $\eta$?

*If you have time:* Study systematically the variance and bias of $\widehat{\Phi}_{\eta, T}$ as a function of $\eta$ and $T$.


In [ ]:
β = 2.
Δt = 0.01
T_final = 1000

ηs = 0.05:0.1:0.55
Ts = [100, 250, 500, 750, 1000.] 
Ts_inds = floor.(Int64, Ts ./ Δt)
num_runs = 500

phi_hats = zeros(length(ηs), length(Ts), num_runs)

for (i, η) in enumerate(ηs)
    #Do the runs in parallel using multi-threading (if you are using the multi-threaded kernel)
    Threads.@threads for k in 1:num_runs
        #We specify local scope for the following variable to avoid conflicts between treads
        local RNG = Xoshiro(k*1000 + i)
        local x_0 = randn(RNG, 2) ./ sqrt(β)
        local _, x_traj = overdamped_trajectory(x_0, η, β, Δt, T_final, ∇Harmonic, rotation; RNG = RNG, verbose = false)
        
        
        phi_hats[i, :, k] .= (cumsum(R.(x_traj[:,1], x_traj[:,2])) ./ axes(x_traj, 1) ./η)[Ts_inds]
    end
end

phi_hat_vars = var(phi_hats; dims = 3)[:, :, 1]
phi_hat_bias = mean(phi_hats; dims = 3)[:, :, 1] .- α_R(β)
;

In [ ]:
plot(yscale = :log10, xscale = :log10, xlabel = "η", title = "Variance of standard estimator")
for (i, T) in enumerate(Ts)
    plot!(ηs, phi_hat_vars[:, i], labels = "T = $(T)")
end
plot!()

In [ ]:
plot(yscale = :log10, xscale = :log10, xlabel = "η", title = "Bias of standard estimator")
for (i, T) in enumerate(Ts)
    plot!(ηs, abs.(phi_hat_bias)[:, i], labels = "T = $(T)")
end
plot!()

Are the variance and bias monotonic function of $\eta$? Why?

What about the bias due to finite integration time, i.e.
$$ \text{Finite Integration Time Bias} := \widehat{\Phi}_{\eta,T} - \alpha_{\eta, R}.$$

In [ ]:
phi_hat_finite_time_bias = vcat([mean(phi_hats; dims = 3)[i, :, 1] .- α_η_R(β,η) for (i,η) in enumerate(ηs)]'...);

In [ ]:
plot(yscale = :log10, xscale = :log10, xlabel = "η", title = "Finite Integration Time Bias of standard estimator")
for (i, T) in enumerate(Ts)
    plot!(ηs, abs.(phi_hat_finite_time_bias)[:, i], labels = "T = $(T)")
end
plot!()

#### ii) Entropic switch potential 

As our second example we consider the entropic switch potential introduced yesterday
$$
\tag{Entropic switch}
V(x,y)=3{\rm e}^{-x^2}({\rm e}^{-(y-1/3)^2}-{\rm e}^{-(y-5/3)^2})-5{\rm e}^{-y^2}({\rm e}^{-(x-1)^2}+{\rm e}^{-(x+1)^2})+0.2x^4+0.2(y-1/3)^4.
$$

As an observable, we consider the first component, $R(x) = x_1$. By the symmetry in the $x$ direction of the potential, $\nu_0(R) = 0$. 

In [ ]:
"""
    entropic_switch(x, y)

Potential energy function.
"""
function entropic_switch(x, y)
    tmp1 = x^2
    tmp2 = (y - 1 / 3)^2
    return 3 * exp(-tmp1) * (exp(-tmp2) - exp(-(y - 5 / 3)^2)) - 5 * exp(-y^2) * (exp(-(x - 1)^2) + exp(-(x + 1)^2)) + 0.2 * tmp1^2 + 0.2 * tmp2^2
end

function entropic_switch(q::Vector{<:Real})
    return entropic_switch(q...)
end

"""
    ∇entropic_switch(x, y)

Gradient of the potential energy function.
"""
function ∇entropic_switch(x, y)

    tmp1 = exp(4*x)
    tmp2 = exp(-x^2 - 2*x - y^2 - 1)
    tmp3 = exp(-x^2)
    tmp4 = exp(-(y-1/3)^2)
    tmp5 = exp(-(y-5/3)^2)

    dx = 0.8*x^3 + 10*(tmp1*(x - 1) + x + 1)*tmp2 - 6*tmp3*x*(tmp4 - tmp5)

    dy = 10*(tmp1 + 1)*y*tmp2 + 3*tmp3*(2*tmp5*(y - 5/3) - 2*tmp4*(y - 1/3)) + 0.8*(y - 1/3)^3

    return [dx, dy]
end

function ∇entropic_switch(q::Vector{<:Real})
    return ∇entropic_switch(q...)
end

xlims = -2.5, 2.5
ylims = -1.75, 2.5
xrange = range(xlims..., 200)
yrange = range(ylims..., 200)
qₗ = (-1.048, -0.04210, Plots.text(L"$q_{\rm L}$", pointsize=25))
qᵣ = (1.048, -0.04210, Plots.text(L"$q_{\rm R}$", pointsize=25))
qₛ = (0., 1.5371, Plots.text(L"$q_{\rm S}$", pointsize=25))
pl_V = contourf(xrange, yrange, entropic_switch, levels=50, cmap=:hsv)
annotate!(qₗ)
annotate!(qᵣ)
annotate!(qₛ)

In the next cell, we plot a trajectory of the discretized estimator $\widehat{\Phi}_{\eta, t}$.

Vary the total runtime $T$ and strength of the perturbation $\eta$ and comment on the convergence of $\widehat{\Phi}_{\eta, t}$. For this potential, it is also interesting to vary the inverse temperature $\beta$.

In [ ]:
x_0 = [1., 0]
η = 0.1
β = 2
Δt = 0.01
T = 50000
burn = 1000
RNG = Xoshiro(24092023) #fixing RNG


#Burning in the dynamics
x, _ = overdamped_trajectory(x_0, η, β, Δt, burn, ∇entropic_switch, rotation;  RNG = RNG, verbose = false)

_, x_traj = overdamped_trajectory(x, η, β, Δt, T, ∇entropic_switch, rotation;  RNG = RNG, verbose = false)
Φ = cumsum(x_traj[:,1]) ./ axes(x_traj, 1) ./η;

In [ ]:
contourf(xrange, yrange, entropic_switch, levels=50, cmap=:hsv)
plot!(x_traj[1:5:10000, 1], x_traj[1:5:10000, 2], color=:black, label = "")

In [ ]:
plot(0.:50*Δt:T, Φ[1:50:end], label = L"\hat{\Phi}_{\eta, T}")
plot!(legendfontsize = 15)
plot!(ylims = (-3., 3.))

What do you observe about the variance and bias of the estimator as a function of $\eta$? What do you think might be the effect of the temperature in this case?


## 2) Kinetic Case

We now consider perturbed kinetic Langevin dynamics:
$$
\tag{Non-equilibrium Langevin}
\left\lbrace
\begin{aligned}
{\rm d}q_t^\eta &= M^{-1}p_t^\eta\,{\rm d}t,\\
{\rm d}p_t^\eta &= \left(-\nabla V\left(q_t^\eta\right)\, + \eta F\left(q_t^\eta\right)\right){\rm d}t -\gamma M^{-1}p_t^\eta\,{\rm d}t+\sqrt{\frac{2\gamma}{\beta}}{\rm d}W_t,
\end{aligned}
\right.
$$
As before, under appropriate assumptions on $V$ and $F$, this dynamics admits a unique invariant probabilty measure $\nu_{\eta}$ with this measure being the Boltzmann-Gibbs measure when $\eta = 0$
$$ \nu_0\left({\rm d}q\, {\rm d}p\right) = Z_\beta^{-1} \exp\left(-\beta H(q,p)\right){\rm d}q\,{\rm d}p.$$
Here $H(q,p) = V(q) +\frac{1}{2}p^{\mathsf{T}} M^{-1}p$ is the dynamics Hamiltonian (see this morning's notebook). To simplfy the presentation we take the mass matrix to be the identity $M = \mathrm{Id}$.

The previous discussion of transport coefficents as well as the NEMD estimator and its error carries over _mutatis mutandis_.

In this subsection (the kinetic case), we choose as our observable the mobility in the direction of the rotational forcing for the two potentials
$$ R(q,p) = F(q)^{\mathsf{T}}M^{-1}p$$
We could have also chosen the mobility in some fixed direction $v$: $\ R(q,p) = v^{\mathsf{T}}M^{-1}p$.

In [ ]:
mobility(q,p) = dot(rotation(q), p);

To discretize this dynamics we modify the BAOAB scheme seen this morning by adding the perturbation to the B step.

<b>Task</b>: Modify the `BAOAB!` function from this morning to make non-equilibirium version

<i>Hint</i>: The argument `force_field` plays the same role as `∇V0` in the function `BAOAB!`. How do we compute this vector at each step?

In [ ]:
"""
    non-eq_BAOAB!(q::Vector{<:Real}, p::Vector{<:Real}, η, Δt, β, γ, force_field::Vector{<:Real}, ∇V::Function, F::Function)

BAOAB integrator for the Langevin dynamics. (q,p) is the initial configuration, Δt the time step, β the inverse temperature, γ the friction parameter, ∇V the negative force field, ∇V the gradient of V evaluated at (q,p). The Ornstein-Uhlenbeck process is integrated using the analytical expression of the solution.

The configuration and the gradient of the potential are updated in place.
"""
function noneq_BAOAB!(q::Vector{<:Real}, p::Vector{<:Real}, η, Δt, β, γ, force_field::Vector{<:Real}, ∇V::Function, F::Function; 
        RNG = Random.default_rng())
        @. p -= Δt * force_field / 2
        @. q += Δt * p / 2
        p .= exp(-γ*Δt) .* p .+ sqrt((1-exp(-2*γ*Δt))/β) .* randn(size(p))
        @. q += Δt * p / 2
        force_field .= ∇V(q) .+ η .* F(q)
        @. p -= Δt * force_field / 2

    return nothing
end

"""
   overdamped_trajectory(x_0::Vector{<:Real}, η, β, Δt, ∇V::Function, F::Function; RNG = Random.default_rng(), verbose::Bool=true)

Simulate overdamped trajectory starting from `x_0` with an Euler-Maruyama scheme using time step `Δt` and total 
integration time `T`. The random number generator can be specified via optional keyword argument `RNG`.

Outputs final position and trajectory
"""

function kinetic_trajectory(q_0::Vector{<:Real}, p_0::Vector{<:Real}, η, β, γ, Δt, T, ∇V::Function, F::Function; 
        RNG = Random.default_rng(), verbose::Bool=true)    
    # Get the number of iterations
    Nit = round(Int, T/Δt)
    Δt = T/Nit
    
    # Frequency of the points to show the progress of the simulations
    nlog = round(Int, Nit/10)

    #Initialize arrays holding position and momentum trajectory
    positions = Array{Float64, 2}(undef, Nit+1, length(q_0))
    momenta = Array{Float64, 2}(undef, Nit+1, length(p_0))
    positions[1, :] .= q_0
    momenta[1, :] .= p_0

    #Initial position and momentum
    q = copy(q_0)
    p = copy(p_0)

    #Initial value of forced field
    force_field = ∇V(q) .+ η .* F(q) #TODO what should be here?

    if verbose
        print("Integrating using the kinetic dynamics for $(Nit) iterations\t"); flush(stdout)
    end

    for i in 2:(Nit+1)
        
        # Prints a point every nlog iterations
        if verbose && i % nlog == 0
            print("."); flush(stdout)
        end

        # Integrate one step
        noneq_BAOAB!(q, p, η, Δt, β, γ, force_field, ∇V, F; RNG = RNG)

        # Update the trajectory
        positions[i,:] = q
        momenta[i,:] = p
    end
    
    if verbose
        println("\tDone"); flush(stdout)
    end

    return q, p, positions, momenta

end
;

#### i) Harmonic potential (Ornstein-Uhlenbeck process)


In [ ]:
#Parameters of the sample trajectory feel free to modify and observe what changes
q_0 = [1., 0]
p_0 = zeros(2)
η = 0.05
β = 2.
γ = 1.
Δt = 0.01
T = 50000
burn = 1000
RNG = Xoshiro(24092023) #fixing RNG

#burn in dynamics
q, p, _, _ = kinetic_trajectory(q_0, p_0, 0, β, γ, Δt, burn, ∇Harmonic, rotation; RNG = RNG, verbose=false)
#simulate a trajectory
_, _, positions, momenta = kinetic_trajectory(q, p, η, β, γ, Δt, T, ∇Harmonic, rotation; RNG = RNG, verbose=false)
Φ = cumsum([mobility(positions[i,:], momenta[i,:]) for i in axes(positions, 1)]) ./ axes(positions, 1) ./η;

In [ ]:
contourf(xrange, yrange, Harmonic, levels=50, cmap=:hsv)
plot!(positions[1:10000, 1], positions[1:10000, 2], color=:black, label = "")

In [ ]:
plot(0.:50*Δt:T, Φ[1:50:end], label = L"\widehat{\Phi}_{\eta, T}")
plot!(legendfontsize = 15, ylims = (-0.5, 0.5))

#### ii) Entropic switch potential 


In [ ]:
#Parameters of the sample trajectory feel free to modify and observe what changes
q_0 = [1., 0]
p_0 = zeros(2)
η = 0.05
β = 1.
γ = 1.
Δt = 0.01
T = 100000
burn = 1000
RNG = Xoshiro(24092023) #fixing RNG


#Burning in the dynamics
q, p, _, _ = kinetic_trajectory(q_0, p_0, η, β, γ, Δt, burn, ∇entropic_switch, rotation; RNG = RNG, verbose=false)  

#simulate a trajectory
_, _, positions, momenta = kinetic_trajectory(q, p, η, β, γ, Δt, T, ∇entropic_switch, rotation; RNG = RNG, verbose=true)
Φ = cumsum([mobility(positions[i,:], momenta[i,:]) for i in axes(positions, 1)]) ./ axes(positions, 1) ./η;

In [ ]:
contourf(xrange, yrange, entropic_switch, levels=50, cmap=:hsv)
plot!(positions[1:20000, 1], positions[1:20000, 2], color=:black, label = "")

In [ ]:
plot(0.:50*Δt:T, Φ[1:50:end], label = L"\widehat{\Phi}_{\eta, T}")
plot!(legendfontsize = 15, ylims = (-0.8, 0.8))

## II) (Probabilistic) Couplings

In this section, we consider couplings of diffusions and their discretization via their driving noise. These couplings are Markovian in the sense that the coupled process is a Markov process. We concentrate on the overdamped case and only consider the kinetic case in an exercise at the end.

In this section we look couplings in two case:
#### a) Starting from different inital conditions
We can couple two solutions to an SDE with different initial conditions. Assuming deterministic initial conditions, a pair of coupled solutions to an overdamped Langevin equation with different initial conditions satisfies:
$$\tag{Same Drift}
\begin{aligned}
    dX_t^0 &= -\nabla V\left(X_t^0\right)dt + \sqrt{\frac{2}{\beta}}dW_t, \qquad X^0_0 = x,\\
    dY_t^0 &= -\nabla V\left(Y_t^0\right)dt + \sqrt{\frac{2}{\beta}}d\widetilde{W}_t, \qquad Y^0_0 = y,
\end{aligned}
$$
with $x, y \in \mathbb{R}^d$ $x\neq y$ and $(W, \widetilde{W})$ a cleverly coupled pair of Brownian motions. In this case, the coupling can help us see how quickly the system mixes/converges to its invariant measure/thermalizes (the terminology depends on the field).

#### b) Different drifts
We can alternatively couple solutions to two SDEs with different drifts. For example, we can couple the non-equilibriun ($\eta \neq 0$) and equilibrium ($\eta = 0$):
$$\tag{Different Drifts}
\begin{aligned}
    dX_t^\eta &= \left(-\nabla V\left(X_t^\eta\right) + \eta F\left(X_t^\eta\right)\right)dt + \sqrt{\frac{2}{\beta}}dW_t, \qquad &X_0 = x\\
    dY_t^0 &= -\nabla V\left(Y_t\right)dt + \sqrt{\frac{2}{\beta}}d\widetilde{W}_t, &Y_0 = y,
\end{aligned}
$$
with $x, y \in \mathbb{R}^d$ and $(W, \widetilde{W})$ a cleverly coupled pair of Brownian motions. Here, the coupling can help us isolate the effect of the perturbation from the effect of the noise. 


To improve upon the estimator from the previous section, we can consider the following coupling based estimator
$$\widehat{\Psi}_{\eta, T} = \frac{1}{\eta T}\int_0^T\left[R\left(X_t^\eta\right) - R\left(Y_t^0\right)\right]dt,$$
with $X^\eta$ and $Y^0$ coupled via their driving noise $(W, \widetilde{W})$.
If the coupling is chosen in such a way that the two trajectories stay close for a long time, we can then hope that the estimator $\widehat{\Psi}_{\eta, T}$ has lower variance than that of $\widehat{\Phi}_{\eta, T}$.

<details>
    <summary>Some theory</summary>
    A coupling of two random variables \(X\) and \(Y\) is a couple \(\left(\widetilde{X}, \widetilde{Y}\right)\) of random variables such that \(\widetilde{X} \stackrel{\mathrm{Law}}{=} X\) and \(\widetilde{Y} \stackrel{\mathrm{Law}}{=} Y\). In terms of probability measures, a coupling of two probability measures $\mu$ on $\left(\mathcal{X}, \mathcal{B}\left(\mathcal{X}\right)\right)$ and $\nu$ on $\left(\mathcal{Y}, \mathcal{B}\left(\mathcal{Y}\right)\right)$ is a probability $\pi$ on the product space $\left(\mathcal{X}\times\mathcal{Y}, \mathcal{B}\left(\mathcal{X}\times\mathcal{Y}\right)\right)$ such that for all $A \in \mathcal{B}\left(\mathcal{X}\right)$ and $B \in \mathcal{B}\left(\mathcal{Y}\right)$
    $$\pi\left(A\times \mathcal{Y}\right) = \mu\left(A\right), \qquad \pi\left(\mathcal{X}\times B\right) = \nu\left(B\right).$$
    We denote by $\Pi(\mu,\nu)$ the set of couplings of $\mu$ and $\nu$.
    Two commonly used distances on the space of probability measures on $\mathbb{R}^d$ the $p$-Wasserstein and the total variation distances can be expressed as the infimum over coupling:
    $$\tag{$p$-Wasserstein}
    \mathcal{W}_p\left(\mu,\nu\right)^p := \inf_{\pi \in \Pi(\mu,\nu)} \int_{\mathbb{R}^d \times \mathbb{R}^d} \left|x-y\right|^p \pi({\rm d}x\, {\rm d} y),$$
    $$\tag{Total variation}
    d_{\rm TV}\left(\mu, \nu\right) := \inf_{\pi \in \Pi(\mu,\nu)} \int_{\mathbb{R}^d \times \mathbb{R}^d} \mathbf{1}_{\left\{x \neq y\right\}} \pi({\rm d}x\, {\rm d} y).
    $$
    Consequently, an explicit coupling gives us an upper bound these distance. Couplings can be used to estimate the contraction rate, i.e. how quickly the dynamics forgets its initial conditions, of the diffusion semi-group, $\left(P_t\right)_{t\geq 0}$, corresponding to an SDE in Wasserstein or total variation distance:
    $$\mathcal{W}_p\left(\delta_xP_t, \delta_yP_t\right) \leq \mathbb{E}_{(x,y)}\left[\left|X_t - Y_t\right|^p\right]^{1/p}, $$
    or 
    $$ d_{\mathrm{TV}}\left(\delta_xP_t, \delta_yP_t\right) \leq \mathbb{P}_{(x,y)}\left(X_t\neq Y_t\right),$$
    with $\left(X_t,Y_t\right)_{t\geq 0}$ a coupling of the solutions to the SDE corresponding to $\left(P_t\right)_{t\geq 0}$ with inital conditions $X_0 = x$ and $Y_0 = y$.
    Couplings can also be used bound the distance between the laws of two diffusions with different drifts. Denote by $\left(P_t^\eta\right)_{t\geq 0}$ the semi-group corresponding to $(\text{Non-equilibrium dynamics})$ for $\eta \in \mathbb{R}$, then
    $$\mathcal{W}_p\left(\delta_xP_t^\eta, \delta_yP_t^0\right) \leq \mathbb{E}_{(x,y)}\left[\left|X_t^\eta- Y_t^0\right|^p\right]^{1/p}, $$
    or 
    $$ d_{\mathrm{TV}}\left(\delta_xP_t^\eta, \delta_yP_t^0\right) \leq \mathbb{P}_{(x,y)}\left(X_t^\eta\neq Y_t^0\right),$$
    with $\left(X_t^\eta,Y_t^0\right)_{t\geq 0}$ a coupling of the nonwith inital conditions $X_0^\eta = x$ and $Y_0^0 = y$.
    \
    \
    Turning to the second moment (which is a proxy for the variance) of the coupling based estimator
</details>

<br>

A simple way to couple the dynamics is to <i>synchronously</i> couple them, i.e. to drive them with the same Brownian motion, $W = \widetilde{W}$. Two possible ways to implement this: 

1. Integrate the coupled dynamics explicitly by sampling at each step $G_k \sim \mathcal{N}\left(0, \mathrm{Id}\right)$ and using the same $G_k$ for the two trajectories:
       $$\begin{aligned}
           X_{k}^\eta &= X_{k-1}^\eta + \Delta t\left(-\nabla V\left(X_{k-1}^\eta\right) + \eta F\left(X_{k-1}^\eta\right)\right) + \sqrt{\frac{2\Delta t}{\beta}} G_k\\
           Y_k^0 &= Y_{k-1}^0 - \Delta t \nabla V\left(Y_{k-1}^0\right) + \sqrt{\frac{2\Delta t}{\beta}} G_k
       \end{aligned}
       $$
3. Integrate the whole trajectory of each dynamics one after the other and starting them both with the same seed.

The first method creates code that can easily be modified to use other more intricate coupling methods. While the second is easy to implement in existing code. To see both methods, we will use the first for the overdamped dynamic and the second for the kinetic dynamics.

### 1) Synchronously coupled overdamped dynamics

<b>Instructive optional exercise: </b> Consider the system (Same Drift) given above with an $m$-strongly convex potential $V$ with $m > 0$, i.e. $\nabla^2 V(x) \geq m \mathrm{Id}$ in the sense of positive definite matricies for all $x \in \mathbb{R}^d$. Compute the differential of $\left|X_t - Y_t\right|^2$ when $X$ and $Y$ are synchronously coupled. What can we say about the behavior of $\left|X_t - Y_t\right|^2$ with time?

<i>Hint 1:</i> If you do not know Itô Calculus, don't worry! In this case the Itô correction terms cancel out, so you can formally use the standard chain rule. If you do know Itô Calculus, why does this happen?
    
<i>Hint 2:</i> Where do you use that fact two equations are synchronously coupled? Where do you use that the fact $V$ is $m$-strongly convex? In case you have not seen this fact before, the $m$-strong convexity of $V$ implies
$$\forall x,y \in \mathbb{R}^d, \qquad \left\langle x-y, \nabla V(x) - \nabla V(y)\right\rangle \geq m\left|x-y\right|^2. $$

<u>Extension</u>: What happens when we consider the system (Different Drifts) and compute the differential of $\left|X_t^\eta - Y_t^0\right|^2$?

In [ ]:
"""
   coupled_traj(drift_x, drift_y, coupled_integrator, β, x_0, y_0, T, burn, Δt; RNG = Random.default_rng())

A wrapper function for simulating trajectories of two discretized diffusions coupled via their noises. The 
argument `coupled_integrator` should be a function that implements the one step iteration of the coupled dynamics.
signature `coupled_integrator(x::Vector{<:Real}, y::Vector{<:Real}, drift_x::Function, drift_y::Function, β, Δt, RNG)`

"""
function coupled_traj(drift_x, drift_y, coupled_integrator, β, x_0, y_0, T, burn, Δt; RNG = Random.default_rng()) 
    Nit= floor(Int64, T/Δt)
    Δt = T/Nit
    K = floor(Int64, burn/Δt)
    
    x_traj = Array{Float64, 2}(undef, Nit+1, length(x_0))
    y_traj = Array{Float64, 2}(undef, Nit+1, length(x_0))

    x = copy(x_0)
    y = copy(y_0)
    
    for _ in 1:K
        #"burning-in" the dynamics. Set `burn` to zero if you do not 
        #want a burn-in
        coupled_integrator(x, y, drift_x, drift_y, β, Δt, RNG)
    end
    
    x_traj[1, :] .= x
    y_traj[1, :] .= y
    for i in 2:(Nit+1)
        coupled_integrator(x, y, drift_x, drift_y, β, Δt, RNG)
        x_traj[i, :] .= x
        y_traj[i, :] .= y
    end
    
    return x_traj, y_traj
end
;

<b>Task:</b> Implement one integration step of synchronously coupled dynamics in the next cell.

In [ ]:
"""
    synchronous_integration_step!(x,y, drift_x, drift_y, β, dt, RNG)
One step of the synchronously coupled dynamics
"""
function synchronous_integration_step!(x,y, drift_x, drift_y, β, dt, RNG)
    G = randn(RNG, length(x))
    x .+= drift_x(x) .* dt .+ sqrt(2*dt/β) .* G
    y .+= drift_y(y) .* dt .+ sqrt(2*dt/β) .* G
    return nothing
end
;

#### a) Strongly Convex Harmonic Potential

Let us test your implementation of synchronous coupling on the overdamped Langevin dynamics with a harmonic potential introduced in the first section.

We start by looking at the convergence towards one another of two coupled solutions to the equilibrium dynamics with different initial conditions. 

By passing to the coordinate system induced by the eigenvectors of $A$, we can solve explicitly for the behavior of the difference process $X_t - Y_t$ since our SDE is linear. If you want to do this, the eigenvalues of $A$ are $\lambda_1 = 1$ and $\lambda_2 = 3$ and its eigenvectors are
$$v_1 = \frac{1}{\sqrt{2}}\left[ \begin{matrix} 1\\ 1\end{matrix}\right], \qquad v_2 = \frac{1}{\sqrt{2}}\left[\begin{matrix} 1\\ -1\end{matrix}\right]. $$

In [ ]:
drift(x) = -∇Harmonic(x)

B = [sqrt(2) 0
    sqrt(2)/2 sqrt(6)/2]

RNG = Xoshiro(24092023) #fixing RNG

###################################
#two samples from the invariant measure of the continuous-time process
x_0 = sqrt(1/(3β)) * B * randn(RNG, 2)
y_0 = sqrt(1/(3β)) * B * randn(RNG, 2)
####################################


# ###################################
# #Difference in the first eigendirection
# x_0 = [-1., 1]
# y_0 = zeros(2)
# ####################################

# ###################################
# #Difference in the second eigendirection
# x_0 = [1., 1]
# y_0 = zeros(2)
# ####################################

T = 5.
dt = 0.01
β = 1.
burn = 0.
x_traj, y_traj = coupled_traj(drift, drift, synchronous_integration_step!, β, x_0, y_0, T, burn, dt; RNG = RNG)
;

Below we plot the decay of the difference between the two solutions in the two eigen-directions of $A$. The decay of the difference in each eigen-direction should behave like $\mathrm{e}^{-\lambda t}$ with $\lambda$ the corresponding eigenvalue.

In [ ]:
#Change of coordinates 
u(x, y) = (x + y)/sqrt(2)
v(x, y) = (x - y)/sqrt(2)
u(x::Vector{<:Real}) = u(x...)
v(x::Vector{<:Real}) = v(x...)

plot(yscale = :log10)
scatter!(0.:10dt:T, [abs(u(x_traj[i, :] - y_traj[i, :])) for i in axes(x_traj, 1)][1:10:end],
    ms = 2, msw = 0.1, label = "Difference in 1st Eigendirection")
plot!(0.:dt:T, abs(u(x_0 - y_0))*exp.(.-(0.:dt:T)), label = L"\exp{(-t)}", lw = 1.5)
scatter!(0.:10dt:T, [abs(v(x_traj[i, :] - y_traj[i, :])) for i in axes(x_traj, 1)][1:10:end], 
    ms = 2, msw = 0.1, label = "Difference in 2nd Eigendirection")
plot!(0.:dt:T, abs(v(x_0 - y_0))*exp.(-3 .*(0.:dt:T)), label = L"\exp{(-3t)}", lw = 1.5)
plot!(legend = :bottomleft, legendfontsize = 12)

We next look at the behavior of the synchronous coupling based estimator of the transport coefficient.

<b> Task:</b> In the next cell vary the intensity of the perturbation $\eta$ and runtime $T$ and comment on the convergence of $\widehat{\Psi}_{\eta,t}^{\mathrm{sync}}$. You may also look at the plots of the trajectories and the distribution of distances between the trajectories in the two cells after that. 

In [ ]:
x_0 = [1., 0]
η = 0.01
β = 1.
Δt = 0.01
T = 5000
burn = 1000
RNG = Xoshiro(24092023) #fixing RNG

drift_x(x) = -∇Harmonic(x) + η*rotation(x) #perturbed drift
drift_y(x) = -∇Harmonic(x) #equilibrium drift

x_traj, y_traj = coupled_traj(drift_x, drift_y, synchronous_integration_step!, β, x_0, y_0, T, burn, Δt; RNG = RNG)
Ψ_sync = cumsum(R.(x_traj[:, 1], x_traj[:, 2]) - R.(y_traj[:, 1], y_traj[:, 2])) ./ axes(x_traj, 1) ./η;

In [ ]:
plot(0:10dt:T, Ψ_sync[1:10:end], label = L"\widehat{\Psi}_{\eta, t}")
plot!([0, T], [α_R(β), α_R(β)], label = L"\alpha_R", legendfontsize = 15)
plot!([0, T], [α_η_R(β, η), α_η_R(β, η)], label = L"\alpha_{\eta,R}")

In [ ]:
#Plotting the trajectories the equilibrium and nonequilibrium processes 
#if you can't differentiate the two trajectories increase η and rerun the previous cell
contourf(xrange, yrange, Harmonic, levels=50, cmap=:hsv)
plot!(x_traj[1:500, 1], x_traj[1:500, 2], color = :black, lw = 1.5, label = L"X^\eta")
plot!(y_traj[1:500, 1], y_traj[1:500, 2], color = :lightgray, lw = 1.5, label = L"Y^0")
plot!(legendfontsize = 15)

In [ ]:
#Plotting the empirical distribution of distance between the equilibrium and nonequilibrium processes
plot()
plot!(title = latexstring("Distribution of \$\\left|X_t^\\eta - Y_t^0\\right|\$"))
plot!([norm([x_traj[i, :] - y_traj[i, :]]) for i in axes(x_traj, 1)], normed = true, label = "", 
    seriestype = :stephist)
plot!(xlabel = "Distance between trajectories")

*If you have time:* Study the variance and bias of the estimator $\widehat{\Psi}_{\eta,T}^{\mathrm{sync}}$ as a function of $\eta$.

*Suggestion*: Burn-in on the order of $T_{\mathrm{burn}} = 10$ should be sufficient. Try also relatively short integration times around $T = 50-100$.

#### b) Non-convex Entropic Switch Potential

We repeat what we did above for the entropic switch potential.

<i> Question for those who did the exercise:</i> Does the result of the exercise still hold in this case?

In [ ]:
drift(x) = -∇entropic_switch(x)

q_l = [-1.048, -0.04210] #left well
q_r = [1.048, -0.04210] #right well
x_0 = [1., 1]
y_0 = [0., 0.]
β = 1.
T = 7.
burn = 0.
dt = 0.01
RNG = Xoshiro(24092023) #fixing RNG
x_traj, y_traj = coupled_traj(drift, drift, synchronous_integration_step!, β, q_l, q_r, T, burn, dt; RNG = RNG)

plot()
plot!(0:dt:T, [norm([x_traj[i, :] - y_traj[i, :]]) for i in axes(x_traj, 1)], label = L"\left|X_t - Y_t\right|")
plot!(yscale = :log, legendfontsize = 15)

In [ ]:
contourf(xrange, yrange, entropic_switch, levels=50, cmap=:hsv)
plot!(x_traj[1:500, 1], x_traj[1:500, 2], color = :black, lw = 1.5, label = L"X")
plot!(y_traj[1:500, 1], y_traj[1:500, 2], color = :lightgray, lw = 1.5, label = L"Y")
plot!(legendfontsize = 15)

What do you observe about the distance between the two solutions over time?

<br>

As before, we look at the behavior of the synchronous coupling based estimator of the transport coefficient.

<b> Task:</b> In the next cell vary the intensity of the perturbation $\eta$, the inverse temperature $\beta$, and runtime $T$ and comment on the convergence of $\widehat{\Psi}_{\eta,t}^{\mathrm{sync}}$. You may also look at the plots of the trajectories in the cell after and the distribution of distances between the trajectories in the cell after that. 

In [ ]:
x_0 = [0., 0]
η = 0.01
β = 1.
Δt = 0.01
T = 100000
burn = 10000
RNG = Xoshiro(24092023) #fixing RNG

drift_x(x) = -∇entropic_switch(x) + η*rotation(x) #perturbed drift
drift_y(x) = -∇entropic_switch(x) #equilibrium drift

x_traj, y_traj = coupled_traj(drift_x, drift_y, synchronous_integration_step!, β, x_0, y_0, T, burn, dt; RNG = RNG)
Ψ_sync = cumsum(x_traj[:, 1] - y_traj[:, 1]) ./ axes(x_traj, 1) ./η;

In [ ]:
plot(0:10dt:T, Ψ_sync[1:10:end], label = L"\widehat{\Psi}_{\eta, t}")
plot!(legendfontsize = 15, ylims = (0, 1))

In [ ]:
contourf(xrange, yrange, entropic_switch, levels=50, cmap=:hsv)
plot!(x_traj[1:500, 1], x_traj[1:500, 2], color = :black, lw = 1.5, label = L"X")
plot!(y_traj[1:500, 1], y_traj[1:500, 2], color = :lightgray, lw = 1.5, label = L"Y")
plot!(legendfontsize = 15)

In [ ]:
#Plotting the empirical distribution of distance between the equilibrium and nonequilibrium processes
plot()
plot!(title = latexstring("Distribution of \$\\left|X_t^\\eta - Y_t^0\\right|\$"))
plot!([norm([x_traj[i, :] - y_traj[i, :]]) for i in axes(x_traj, 1)], normed = true, label = "", 
    seriestype = :stephist)
plot!(xlabel = "Distance between trajectories")

### 2) Synchronously coupled kinetic dynamics

#### a) Strongly Convex Harmonic Potential

Let us try synchronous coupling the kinetic Langevin dynamics by using the same random seed. We start by looking at the convergence towards one another of two coupled solutions to the equilibrium dynamics with different initial conditions. 

*Question for those who did the optional question:* Does the reasoning used to control $\left|X_t^\eta - Y_t^0\right|^2$ still work here?

In [ ]:
B = [sqrt(2) 0
    sqrt(2)/2 sqrt(6)/2]

###################################
#two samples from the invariant measure of the continuous-time process
RNG = Xoshiro(30) #fixing the RNG for sampling the initial conditions
q1_0 = sqrt(1/(3β)) * B * randn(RNG, 2)
q2_0 = sqrt(1/(3β)) * B * randn(RNG, 2)

p1_0 = sqrt(1/(3β)) * randn(RNG, 2)
p2_0 = sqrt(1/(3β)) * randn(RNG, 2)
####################################


# ###################################
# #Difference in the first eigendirection
# q1_0 = [-1., 1]
# q2_0 = zeros(2)

# p1_0 = sqrt(1/(3β)) * randn(RNG, 2)
# p2_0 = sqrt(1/(3β)) * randn(RNG, 2)
# ####################################

# ###################################
# #Difference in the first eigendirection
# q1_0 = [-1., 1]
# q2_0 = zeros(2)

# p1_0 = sqrt(1/(3β)) * randn(RNG, 2)
# p2_0 = sqrt(1/(3β)) * randn(RNG, 2)
# ####################################

T = 50.
Δt = 0.01
β = 1.
γ = 1.
burn = 0.

RNG = Xoshiro(24092023) #fixing RNG for integrating the trajectory
_, _, positions1, momenta1 =  kinetic_trajectory(q1_0, p1_0, 0, β, γ, Δt, T, ∇Harmonic, rotation; RNG =RNG, verbose = false)

RNG = Xoshiro(24092023) #refixing RNG for integrating the trajectory
_, _, positions2, momenta2 =  kinetic_trajectory(q2_0, p2_0, 0, β, γ, Δt, T, ∇Harmonic, rotation; RNG =RNG, verbose = false)
;

In [ ]:
plot()
plot!(0.:Δt:T, [norm(positions1[i, :] - positions2[i, :]) for i in axes(positions1, 1)], label = "Position Difference")
plot!(0.:Δt:T, [norm(momenta1[i, :] - momenta2[i, :]) for i in axes(positions1, 1)], label = "Momentum Difference")
plot!(legendfontsize = 12)

Now we look at the behavior of the kinetic synchronous coupling based estimator of the transport coefficient.

As before, we can explicitly solve everything here since the potential is harmonic.
<details>
    
</details>

<br>

<b> Task:</b> In the next cell vary the intensity of the perturbation $\eta$ and runtime $T$ and comment on the convergence of $\widehat{\Psi}_{\eta,t}^{\mathrm{sync}}$. You may also look at the plots of the trajectories and the distribution of distances between the trajectories in the two cells after that.

In [ ]:
RNG = Xoshiro(30) #fixing the RNG for sampling the initial conditions
q_0 = sqrt(1/(3β)) * B * randn(RNG, 2)
p_0 = sqrt(1/(3β)) * randn(RNG, 2)

T = 5000.
burn = 1000
Δt = 0.01
η = 0.1
β = 1.
γ = 1.

RNG = Xoshiro(24092023) #fixing RNG for integrating the trajectory
#burning in the non-equilibrium dynamics
q, p, _, _ = kinetic_trajectory(q_0, p_0, η, β, γ, Δt, burn, ∇Harmonic, rotation; RNG =RNG, verbose = false)
#simulating a non-equilibrium trajectory
_, _, positions1, momenta1 = kinetic_trajectory(q, p, η, β, γ, Δt, T, ∇Harmonic, rotation; RNG =RNG, verbose = false)

RNG = Xoshiro(24092023) #refixing RNG for integrating the trajectory
#burning in the equilibrium dynamics
q, p, _, _ = kinetic_trajectory(q_0, p_0, 0, β, γ, Δt, burn, ∇Harmonic, rotation; RNG =RNG, verbose = false)
#simulating an equilibrium trajectory
_, _, positions2, momenta2 = kinetic_trajectory(q, p, 0, β, γ, Δt, T, ∇Harmonic, rotation; RNG =RNG, verbose = false)

Ψ = cumsum([mobility(positions1[i,:], momenta1[i,:]) - mobility(positions2[i,:], momenta2[i,:]) for i in axes(positions1, 1)]) ./ axes(positions1, 1) ./η;

In [ ]:
plot(0:10dt:T, Ψ[1:10:end], label = L"\widehat{\Psi}_{\eta, T}")
plot!(legendfontsize = 15)

In [ ]:
#Plotting the empirical distribution of distance between the positions of the equilibrium and nonequilibrium processes
plot()
plot!(title = latexstring("Distribution of \$\\left|q_t^\\eta - q_t^0\\right|\$"))
plot!([norm([positions1[i, :] - positions2[i, :]]) for i in axes(positions1, 1)], normed = true, label = "", 
    seriestype = :stephist)
plot!(xlabel = "Distance between trajectories")

In [ ]:
#Plotting the empirical distribution of distance between the momenta of the equilibrium and nonequilibrium processes
plot()
plot!(title = latexstring("Distribution of \$\\left|p_t^\\eta - p_t^0\\right|\$"))
plot!([norm([momenta1[i, :] - momenta2[i, :]]) for i in axes(momenta1, 1)], normed = true, label = "", 
    seriestype = :stephist)
plot!(xlabel = "Distance between trajectories")

*If you have time:* Study the variance and bias of the estimator $\widehat{\Psi}_{\eta,T}^{\mathrm{sync}}$ as a function of $\eta$.

#### b) Non-convex Entropic Switch Potential

We repeat what we did above for the entropic switch potential.

In [ ]:
RNG = Xoshiro(30) #fixing the RNG for sampling the initial conditions
q1_0 = q_l
q2_0 = q_r

p1_0 = sqrt(1/(3β)) * randn(RNG, 2)
p2_0 = sqrt(1/(3β)) * randn(RNG, 2)

T = 50.
Δt = 0.01
β = 1.
γ = 2.
burn = 0.

RNG = Xoshiro(24092023) #fixing RNG for integrating the trajectory
_, _, positions1, momenta1 =  kinetic_trajectory(q1_0, p1_0, 0, β, γ, Δt, T, ∇entropic_switch, rotation; RNG =RNG, verbose = false)

RNG = Xoshiro(24092023) #refixing RNG for integrating the trajectory
_, _, positions2, momenta2 =  kinetic_trajectory(q2_0, p2_0, 0, β, γ, Δt, T, ∇entropic_switch, rotation; RNG =RNG, verbose = false)
;

In [ ]:
plot()
plot!(0.:Δt:T, [norm(positions1[i, :] - positions2[i, :]) for i in axes(positions1, 1)], label = "Position Difference")
plot!(0.:Δt:T, [norm(momenta1[i, :] - momenta2[i, :]) for i in axes(positions1, 1)], label = "Momentum Difference")
plot!(legendfontsize = 12)

In [ ]:
RNG = Xoshiro(30) #fixing the RNG for sampling the initial conditions
q_0 = q_l
p_0 = sqrt(1/(3β)) * randn(RNG, 2)

T = 10000.
burn = 1000
Δt = 0.01
η = 0.1
β = 1.
γ = 2.

RNG = Xoshiro(24092023) #fixing RNG for integrating the trajectory
#burning in the non-equilibrium dynamics
q, p, _, _ = kinetic_trajectory(q_0, p_0, η, β, γ, Δt, burn, ∇entropic_switch, rotation; RNG =RNG, verbose = false)
#simulating a non-equilibrium trajectory
_, _, positions1, momenta1 = kinetic_trajectory(q, p, η, β, γ, Δt, T, ∇entropic_switch, rotation; RNG =RNG, verbose = false)

RNG = Xoshiro(24092023) #refixing RNG for integrating the trajectory
#burning in the equilibrium dynamics
q, p, _, _ = kinetic_trajectory(q_0, p_0, 0, β, γ, Δt, burn, ∇entropic_switch, rotation; RNG =RNG, verbose = false)
#simulating an equilibrium trajectory
_, _, positions2, momenta2 = kinetic_trajectory(q, p, 0, β, γ, Δt, T, ∇entropic_switch, rotation; RNG =RNG, verbose = false)

Ψ = cumsum([mobility(positions1[i,:], momenta1[i,:]) - mobility(positions2[i,:], momenta2[i,:]) for i in axes(positions1, 1)]) ./ axes(positions1, 1) ./η;

In [ ]:
plot(0:10dt:T, Ψ[1:10:end], label = L"\widehat{\Psi}_{\eta, T}")
plot!(legendfontsize = 15)

In [ ]:
#Plotting the empirical distribution of distance between the positions of the equilibrium and nonequilibrium processes
plot()
plot!(title = latexstring("Distribution of \$\\left|q_t^\\eta - q_t^0\\right|\$"))
plot!([norm([positions1[i, :] - positions2[i, :]]) for i in axes(positions1, 1)], normed = true, label = "", 
    seriestype = :stephist)
plot!(xlabel = "Distance between trajectories")

In [ ]:
#Plotting the empirical distribution of distance between the momenta of the equilibrium and nonequilibrium processes
plot()
plot!(title = latexstring("Distribution of \$\\left|p_t^\\eta - p_t^0\\right|\$"))
plot!([norm([momenta1[i, :] - momenta2[i, :]]) for i in axes(momenta1, 1)], normed = true, label = "", 
    seriestype = :stephist)
plot!(xlabel = "Distance between trajectories")